In [ ]:
from pathlib import Path

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

import holodeck2 as holo
from holodeck2 import physics, utils, cosmo
import holodeck2.constants
from holodeck2.constants import YR

In [ ]:
size = 5
for i in range(3):
    vals = np.random.normal(size=size)
    percs = np.random.uniform(0.0, 1.0, size=size)
    quants = np.quantile(vals, percs)
    data = dict(vals=vals, percs=percs, quants=quants)
    for kk, vv in data.items():
        msg = [f"{v:+.4e}".format(v) for v in vv]
        msg = ", ".join(msg)
        kk = f"{kk}[]"
        msg = f"double {kk:9s} = {{{msg}}}"
        print(msg)

    print()


In [ ]:
def save_cosmo_grid(cosmo, num_redz=2000):
    fname = "cosmology.txt"
    fname = Path(fname).absolute()
    print(f"{fname=}")

    redz = np.logspace(-5, 3, num_redz)
    scafa = cosmo.scale_factor(redz)
    dcom = cosmo.comoving_distance(redz).value
    vcom = cosmo.comoving_volume(redz).value
    tlook = cosmo.lookback_time(redz).to('Myr').value
    efunc = cosmo.efunc(redz)
    legend = (
        f"# H0={cosmo.H0.value:.4e} Om0={cosmo.Om0:.4e} Ob0={cosmo.Ob0:.4e} Ol0={cosmo.Ode0:.4e} | N={num_redz}"
    )
    print(legend)
    with open(fname, 'w') as fout:
        fout.write(legend + "\n")
        fout.write("# redshift scafa dcom[Mpc] vcom[Mpc3] tlook[Myr] efunc\n")
        for i in range(num_redz):
            fout.write(f"{redz[i]:.8e} {scafa[i]:.8e} {dcom[i]:.8e} {vcom[i]:.8e} {tlook[i]:.8e} {efunc[i]:.8e}\n")

    print(f"Saved to file '{fname}'")
    return tlook

tlook = save_cosmo_grid(cosmo)

In [ ]:
# redz = np.power(10.0, sorted(np.random.uniform(-5, 3, 10)))
# test_data = {"redz": redz}
# test_data["scafa"] = cosmo.scale_factor(redz)
# test_data["dcom"] = cosmo.comoving_distance(redz).value
# test_data["vcom"] = cosmo.comoving_volume(redz).value
# test_data["tlook"] = cosmo.lookback_time(redz).to('Myr').value
# test_data["efunc"] = cosmo.efunc(redz)

redz  = [
    1.80422089e-05, 2.34300168e-05, 2.45124079e-05, 2.97102841e-05, 8.72639006e-05,
    4.43186134e-03, 9.82815660e-03, 1.40178590e+00, 6.29892108e+00, 8.64871553e+02
]
scafa = [
    9.99981958e-01, 9.99976571e-01, 9.99975488e-01, 9.99970291e-01, 9.99912744e-01,
    9.95587693e-01, 9.90267496e-01, 4.16356846e-01, 1.37006551e-01, 1.15490571e-03
]
dcom  = [
    7.80166894e-02, 1.01314103e-01, 1.05994463e-01, 1.28470531e-01, 3.77333999e-01,
    1.91456138e+01, 4.24078764e+01, 4.25567300e+03, 8.56800310e+03, 1.39825572e+04
]
vcom  = [
    1.98907501e-03, 4.35610486e-03, 4.98813438e-03, 8.88176278e-03, 2.25043474e-01,
    2.93965587e+04, 3.19468606e+05, 3.22844475e+11, 2.63467744e+12, 1.14511320e+13
]
tlook = [
    2.54454113e-01, 3.30438539e-01, 3.45703464e-01, 4.19008607e-01, 1.23064521e+00,
    6.23067191e+01, 1.37641198e+02, 9.17562386e+03, 1.28651872e+04, 1.37520453e+04
]
efunc = [
    1.00000779e+00, 1.00001012e+00, 1.00001059e+00, 1.00001284e+00, 1.00003770e+00,
    1.00192122e+00, 1.00427848e+00, 2.16845714e+00, 1.06159892e+01, 1.36733979e+04
]

dtdz = cosmo.dtdz(redz)
test_data = dict(dtdz=dtdz)

for kk, vv in test_data.items():
    msg = [f"{v:.8e}".format(v) for v in vv]
    msg = ", ".join(msg)
    kk = f"{kk}[]"
    msg = f"double {kk:7s} = {{{msg}}}"
    print(msg)

In [ ]:
for kk in dir(holodeck2.constants):
    if kk.startswith('__'): continue
    val = getattr(holodeck2.constants, kk)
    print(f"constexpr float {kk:10s} = {val:.8e}")

In [ ]:
sam = holo.sam.SAM()

In [ ]:
edges_3d, ndens_3d = sam.number_density_3d()

In [ ]:
fobs_gw_cents, fobs_gw_edges = physics.pta_freqs()
cents_4d, numb_4d = holo.sam.number_expect_4d_gwonly_instant(fobs_gw_edges, edges_3d, ndens_3d)

In [ ]:
gwb = holo.sam.gws_from_number_expect_instant(fobs_gw_edges, cents_4d, numb_4d, realize=100)

In [ ]:
print(f"{gwb.shape=}")
gwb = np.sqrt(np.sum(gwb, axis=(1, 2, 3)))
print(f"{gwb.shape=}")

plt.loglog(fobs_gw_cents*YR, gwb)
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[12, 5], ncols=3)
labels = ['m1', 'm2', 'z']
ymax = np.sum(ndens_3d)
ymin = ymax / 1e10

for ii, ax in enumerate(axes):
    xx = edges_3d[ii]
    ax.set(
        xscale='log', xlabel=labels[ii],
        yscale='log', ylabel='Density', ylim=[ymin, ymax],
    )

    margin = [0, 1, 2]
    margin.pop(ii)
    yy = np.sum(ndens_3d, axis=tuple(margin))

    ax.plot(xx, yy)

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[12, 10], ncols=3, nrows=3)
labels = ['m1', 'm2', 'z']
ymax = np.sum(ndens_3d)
ymin = ymax / 1e10

for (ii, jj), ax in np.ndenumerate(axes):
    if ii < jj:
        ax.set_visible(False)
        continue

    ax.set(
        xscale='log',
        # xlabel=labels[ii],
        yscale='log',
        # ylim=[ymin, ymax],
        # ylabel='Density',
    )


    xx = edges_3d[jj]

    # ---- 1D
    if ii == jj:
        margin = [0, 1, 2]
        margin.pop(jj)
        yy = np.sum(ndens_3d, axis=tuple(margin))

        ax.plot(xx, yy)
        ax.set(ylim=[ymin, ymax])

    # ---- 2D
    else:
        yy = edges_3d[ii]
        mesh = np.meshgrid(xx, yy, indexing='ij')

        margin = [0, 1, 2]
        margin.pop(ii)  # ii > jj, so do ii first
        margin.pop(jj)
        zz = np.sum(ndens_3d, axis=tuple(margin))

        ax.pcolormesh(*mesh, np.log10(zz), shading='gouraud')



plt.show()